In [ ]:
def on_others_stats_button_clicked(self, b):
        with self.others_stats_output:
            clear_output()
            
            if not hasattr(self.manager, 'data_df') or self.manager.data_df is None:
                print("CSV分類データがありません。インポート/エクスポートタブでCSVを分類すると表示されます。")
                return
            
            try:
                df = self.manager.data_df
                column = self.manager.last_column_name
                
                # 「その他」に分類された項目を集計
                others = df[df['category'] == 'その他']
                others_count = len(others)
                total_count = len(df)
                others_percentage = (others_count / total_count) * 100 if total_count > 0 else 0
                
                # ユニークな値の数を取得
                unique_others = others[column].nunique()
                
                print(f"「その他」に分類: {unique_others:,}種類の語句 / {others_count:,}件")
                print(f"全体に対する割合: {others_percentage:.2f}%")
                
                # 「その他」に分類された上位の例を表示
                if unique_others > 0:
                    print("\n最も多い「その他」の例:")
                    top_others = others[column].value_counts().head(20)
                    for term, count in top_others.items():
                        print(f"  {term}: {count:,}件")
                    
                    # 可視化
                    plt.figure(figsize=(10, 6))
                    sns.barplot(x=top_others.values[:10], y=top_others.index[:10])
                    plt.title('「その他」に分類された上位10件')
                    plt.xlabel('件数')
                    plt.tight_layout()
                    plt.show()
            except Exception as e:
                print(f"統計情報の表示中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                    import json
import os
import re
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import matplotlib.pyplot as plt
import seaborn as sns
import japanize_matplotlib


class CategoryManager:
    """傷病名分類のカテゴリを管理するクラス"""
    def __init__(self, file_path='disease_categories.json'):
        """初期化"""
        self.file_path = file_path
        self.categories = self.load_categories()
        self._compiled_patterns = {}
        self._compile_patterns()
        self.data_df = None
        self.last_csv_path = None
        self.last_column_name = None

    def load_categories(self):
        """カテゴリ定義ファイルを読み込む（順序を維持）"""
        try:
            with open(self.file_path, 'r', encoding='utf-8') as f:
                categories = json.load(f, object_pairs_hook=dict)
            return categories
        except FileNotFoundError:
            print(f"ファイル '{self.file_path}' が見つかりません。空のカテゴリ辞書を作成します。")
            return {}
        except json.JSONDecodeError:
            print(f"ファイル '{self.file_path}' の形式が正しくありません。空のカテゴリ辞書を作成します。")
            return {}
    
    def save_categories(self):
        """カテゴリ定義ファイルを保存する"""
        with open(self.file_path, 'w', encoding='utf-8') as f:
            json.dump(self.categories, f, ensure_ascii=False, indent=2)
        print(f"カテゴリ定義を '{self.file_path}' に保存しました。")
    
    def _compile_patterns(self):
        """正規表現パターンをコンパイルする"""
        self._compiled_patterns = {}
        for category, pattern in self.categories.items():
            try:
                self._compiled_patterns[category] = re.compile(pattern)
            except re.error as e:
                print(f"カテゴリ '{category}' の正規表現エラー: {e}")
    
    def add_category(self, name, pattern):
        """新しいカテゴリを追加する"""
        try:
            re.compile(pattern)
            self.categories[name] = pattern
            self._compile_patterns()
            return True
        except re.error as e:
            print(f"正規表現エラー: {e}")
            return False
    
    def update_category(self, old_name, new_name, new_pattern=None):
        """カテゴリを更新する"""
        if old_name not in self.categories:
            print(f"カテゴリ '{old_name}' は存在しません。")
            return False
        
        # パターンの更新
        if new_pattern is not None:
            try:
                re.compile(new_pattern)
                pattern = new_pattern
            except re.error as e:
                print(f"正規表現エラー: {e}")
                return False
        else:
            pattern = self.categories[old_name]
        
        # 名前の更新
        if new_name != old_name:
            if new_name in self.categories and new_name != old_name:
                print(f"カテゴリ '{new_name}' は既に存在します。")
                return False
            del self.categories[old_name]
        
        self.categories[new_name] = pattern
        self._compile_patterns()
        return True
    
    def delete_category(self, name):
        """カテゴリを削除する"""
        if name not in self.categories:
            print(f"カテゴリ '{name}' は存在しません。")
            return False
        
        del self.categories[name]
        if name in self._compiled_patterns:
            del self._compiled_patterns[name]
        return True
    
    def add_terms_to_category(self, name, terms):
        """カテゴリに語句を末尾に追加する"""
        if name not in self.categories:
            print(f"カテゴリ '{name}' は存在しません。")
            return False
        
        current_pattern = self.categories[name]
        current_terms = current_pattern.strip("()").split("|")
        
        # 新しい語句を分割
        if isinstance(terms, str):
            new_terms = terms.split("|")
        else:
            new_terms = terms
        
        # 語句を末尾に追加（重複は除去せずそのまま追加）
        all_terms = current_terms.copy()
        for term in new_terms:
            if term not in current_terms:  # 既存の語句と重複しない場合のみ追加
                all_terms.append(term)
        
        new_pattern = "(" + "|".join(all_terms) + ")"
        
        # パターンの形式チェック
        try:
            re.compile(new_pattern)
            self.categories[name] = new_pattern
            self._compile_patterns()
            return True
        except re.error as e:
            print(f"正規表現エラー: {e}")
            return False
    
    def test_pattern(self, text, category=None):
        """テキストがパターンにマッチするかテストする"""
        results = []
        
        if category is not None:
            if category not in self._compiled_patterns:
                return [(category, False, "カテゴリが存在しません")]
            pattern = self._compiled_patterns[category]
            match = pattern.search(text)
            return [(category, match is not None, match.group(0) if match else None)]
        
        for cat, pattern in self._compiled_patterns.items():
            match = pattern.search(text)
            results.append((cat, match is not None, match.group(0) if match else None))
        
        return results
    
    def classify(self, text):
        """テキストを最初にマッチするカテゴリに分類する"""
        for cat, pattern in self._compiled_patterns.items():
            if pattern.search(text):
                return cat
        return "その他"
    
    def normalize_disease_name(self, text):
        """傷病名を正規化する"""
        if pd.isna(text):
            return "不明"
            
        # 数値の場合は文字列に変換
        if isinstance(text, (int, float)):
            return str(int(text))
            
        # 文字列の場合
        text = str(text).strip()
        
        # 余分な空白を削除
        text = re.sub(r'\s+', ' ', text)
        
        # 全角記号を半角に
        text = text.replace('（', '(').replace('）', ')')
        text = text.replace('：', ':').replace('，', ',')
        
        # カッコ内の注釈を削除（例: "糖尿病(2型)" → "糖尿病"）
        text = re.sub(r'\([^)]*\)', '', text)
        
        # 先頭の数字や記号を削除
        text = re.sub(r'^[\d\.\,\:\-\+]+\s*', '', text)
        
        # 末尾の句読点を削除
        text = re.sub(r'[、。.]+
    
    def export_to_csv(self, file_path):
        """カテゴリとパターンをCSVに出力する"""
        result = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            for term in terms:
                result.append({"カテゴリ": category, "語句": term})
        
        df = pd.DataFrame(result)
        df.to_csv(file_path, encoding='utf-8', index=False)
        print(f"カテゴリと語句を '{file_path}' に出力しました。")
        return df
    
    def import_from_csv(self, file_path, overwrite=False):
        """CSVからカテゴリとパターンを読み込む"""
        df = pd.read_csv(file_path, encoding='utf-8')
        
        if "カテゴリ" not in df.columns or "語句" not in df.columns:
            print("CSVファイルには 'カテゴリ' と '語句' の列が必要です。")
            return False
        
        # カテゴリごとに語句をグループ化
        new_categories = {}
        for category, group in df.groupby("カテゴリ"):
            terms = group["語句"].tolist()
            pattern = "(" + "|".join(terms) + ")"
            try:
                re.compile(pattern)
                new_categories[category] = pattern
            except re.error as e:
                print(f"カテゴリ '{category}' の正規表現エラー: {e}")
        
        if overwrite:
            self.categories = new_categories
        else:
            self.categories.update(new_categories)
        
        self._compile_patterns()
        print(f"{len(new_categories)}個のカテゴリを '{file_path}' から読み込みました。")
        return True
    
    def get_category_terms(self, name):
        """カテゴリに含まれる語句のリストを取得する（元の順序を維持）"""
        if name not in self.categories:
            return []
        
        pattern = self.categories[name]
        # 括弧を削除し、パイプで区切って語句のリストを取得
        terms = pattern.strip("()").split("|")
        return terms
    
    def get_category_stats(self):
        """カテゴリごとの語句数などの統計情報を取得する"""
        stats = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            stats.append({
                "カテゴリ": category,
                "語句数": len(terms),
                "パターン長": len(pattern)
            })
        
        return pd.DataFrame(stats)


# 簡素化されたインターフェースの実装
class CategoryManagerUI:
    def __init__(self, json_path='disease_categories.json'):
        self.manager = CategoryManager(json_path)
        self.setup_ui()
        
    def setup_ui(self):
        # メインタブを作成
        self.tab = widgets.Tab()
        
        # 各タブの内容を作成
        self.setup_overview_tab()
        self.setup_add_tab()
        self.setup_edit_tab()
        self.setup_test_tab()
        self.setup_import_export_tab()
        
        # タブの設定
        self.tab.children = [
            self.overview_tab,
            self.add_tab,
            self.edit_tab,
            self.test_tab,
            self.import_export_tab
        ]
        
        # タブのタイトル設定
        self.tab.set_title(0, '概要')
        self.tab.set_title(1, 'カテゴリ追加')
        self.tab.set_title(2, 'カテゴリ編集')
        self.tab.set_title(3, 'テスト')
        self.tab.set_title(4, 'インポート/エクスポート')
        
        # UIを表示
        display(self.tab)
        self.refresh_overview()
    
    def setup_overview_tab(self):
        # 更新ボタン
        self.refresh_button = widgets.Button(
            description='更新',
            button_style='info',
            icon='refresh'
        )
        self.refresh_button.on_click(self.on_refresh_button_clicked)
        
        # 保存ボタン
        self.save_button = widgets.Button(
            description='保存',
            button_style='success',
            icon='save'
        )
        self.save_button.on_click(self.on_save_button_clicked)
        
        # 出力エリア
        self.overview_output = widgets.Output()
        
        # タブのレイアウト
        self.overview_tab = widgets.VBox([
            widgets.HBox([self.refresh_button, self.save_button]),
            self.overview_output
        ])
    
    def setup_add_tab(self):
        # 入力フィールド
        self.new_category_name = widgets.Text(
            description='カテゴリ名:',
            style={'description_width': 'initial'}
        )
        
        self.new_category_pattern = widgets.Textarea(
            description='パターン:',
            placeholder='例: (風邪|感冒|発熱)',
            style={'description_width': 'initial'}
        )
        
        # 追加ボタン
        self.add_button = widgets.Button(
            description='追加',
            button_style='success',
            icon='plus'
        )
        self.add_button.on_click(self.on_add_button_clicked)
        
        # 出力エリア
        self.add_output = widgets.Output()
        
        # タブのレイアウト
        self.add_tab = widgets.VBox([
            self.new_category_name,
            self.new_category_pattern,
            self.add_button,
            self.add_output
        ])
    
    def setup_edit_tab(self):
        # カテゴリ選択ドロップダウン
        self.category_dropdown = widgets.Dropdown(
            description='カテゴリ:',
            style={'description_width': 'initial'}
        )
        
        # 読み込みボタン
        self.load_edit_button = widgets.Button(
            description='読み込み',
            button_style='info',
            icon='download'
        )
        self.load_edit_button.on_click(self.on_load_edit_button_clicked)
        
        # カテゴリ名変更
        self.edit_category_name = widgets.Text(
            description='新カテゴリ名:',
            style={'description_width': 'initial'}
        )
        
        self.update_name_button = widgets.Button(
            description='名前変更',
            button_style='warning',
            icon='edit'
        )
        self.update_name_button.on_click(self.on_update_name_button_clicked)
        
        # 語句追加
        self.edit_terms_textarea = widgets.Textarea(
            description='追加する語句:',
            placeholder='例: 喘鳴|鼻汁|鼻閉|くしゃみ（|で区切って入力）',
            style={'description_width': 'initial'}
        )
        
        self.add_terms_button = widgets.Button(
            description='語句追加',
            button_style='success',
            icon='plus'
        )
        self.add_terms_button.on_click(self.on_add_terms_button_clicked)
        
        # 削除ボタン
        self.delete_button = widgets.Button(
            description='削除',
            button_style='danger',
            icon='trash'
        )
        self.delete_button.on_click(self.on_delete_button_clicked)
        
        # 語句一覧表示エリア
        self.terms_list_output = widgets.Output()
        
        # 「その他」の統計情報
        self.others_stats_button = widgets.Button(
            description='「その他」の統計表示',
            button_style='info',
            icon='bar-chart'
        )
        self.others_stats_button.on_click(self.on_others_stats_button_clicked)
        
        self.others_stats_output = widgets.Output()
        
        # 出力エリア
        self.edit_output = widgets.Output()
        
        # タブのレイアウト
        self.edit_tab = widgets.VBox([
            widgets.HBox([self.category_dropdown, self.load_edit_button]),
            widgets.HTML("<h4>カテゴリ名の変更</h4>"),
            widgets.HBox([self.edit_category_name, self.update_name_button]),
            widgets.HTML("<h4>語句の追加</h4>"),
            self.edit_terms_textarea,
            self.add_terms_button,
            widgets.HTML("<h4>カテゴリの削除</h4>"),
            self.delete_button,
            widgets.HTML("<h4>登録語句一覧</h4>"),
            self.terms_list_output,
            widgets.HTML("<h4>「その他」に分類される語句の統計</h4>"),
            self.others_stats_button,
            self.others_stats_output,
            self.edit_output
        ])
        
        # カテゴリリストを更新
        self.update_category_dropdown()
    
    def setup_test_tab(self):
        # テストテキスト入力
        self.test_text = widgets.Text(
            value='',
            placeholder='例: 頭痛、発熱、咳、腹痛など',
            description='テストテキスト:',
            style={'description_width': 'initial'}
        )
        
        # テストボタン
        self.test_button = widgets.Button(
            description='テスト',
            button_style='info',
            icon='search'
        )
        self.test_button.on_click(self.on_test_button_clicked)
        
        # 出力エリア
        self.test_output = widgets.Output()
        
        # タブのレイアウト
        self.test_tab = widgets.VBox([
            self.test_text,
            self.test_button,
            self.test_output
        ])
    
    def setup_import_export_tab(self):
        # ファイル名入力
        self.csv_file_path = widgets.Text(
            value='disease_categories.csv',
            description='CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # エクスポートボタン
        self.export_button = widgets.Button(
            description='エクスポート',
            button_style='primary',
            icon='file-export'
        )
        self.export_button.on_click(self.on_export_button_clicked)
        
        # インポートボタン
        self.import_button = widgets.Button(
            description='インポート',
            button_style='info',
            icon='file-import'
        )
        self.import_button.on_click(self.on_import_button_clicked)
        
        # インポート方法
        self.import_mode = widgets.RadioButtons(
            options=[('新規カテゴリのみ追加', 'add'), ('すべて上書き', 'overwrite')],
            value='add',
            description='インポート方法:',
            style={'description_width': 'initial'}
        )
        
        # 出力エリア
        self.import_export_output = widgets.Output()
        
        # バッチ処理セクション
        self.batch_title = widgets.HTML("<h3>CSVファイルの傷病名を一括分類</h3>")
        
        # 入力CSVファイル
        self.input_csv_file = widgets.Text(
            value='',
            placeholder='例: 傷病名一覧.csv',
            description='入力CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # 傷病名の列名
        self.disease_column = widgets.Text(
            value='傷病名',
            description='傷病名の列:',
            style={'description_width': 'initial'}
        )
        
        # 出力CSVファイル名
        self.output_csv_file = widgets.Text(
            value='classified_diseases.csv',
            description='出力CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # 分類実行ボタン
        self.classify_button = widgets.Button(
            description='分類実行',
            button_style='primary',
            icon='play'
        )
        self.classify_button.on_click(self.on_classify_button_clicked)
        
        # バッチ処理の出力エリア
        self.batch_output = widgets.Output()
        
        # タブのレイアウト
        self.import_export_tab = widgets.VBox([
            widgets.HTML("<h3>カテゴリの入出力</h3>"),
            self.csv_file_path,
            widgets.HBox([self.export_button, self.import_button]),
            self.import_mode,
            self.import_export_output,
            widgets.HTML("<hr>"),
            self.batch_title,
            self.input_csv_file,
            self.disease_column,
            self.output_csv_file,
            self.classify_button,
            self.batch_output
        ])
    
    # ========== イベントハンドラ ==========
    def on_refresh_button_clicked(self, b):
        self.refresh_overview()
    
    def refresh_overview(self):
        with self.overview_output:
            clear_output()
            
            # カテゴリ数を表示
            categories_count = len(self.manager.categories)
            print(f"現在のファイル: {self.manager.file_path}")
            print(f"{categories_count}個のカテゴリが登録されています。")
            
            # 語句の総数を計算
            total_terms = 0
            for pattern in self.manager.categories.values():
                terms = pattern.strip("()").split("|")
                total_terms += len(terms)
            
            print(f"登録されている語句の総数: {total_terms}語")
            
            # 各カテゴリの語句数の統計
            print("\n=== カテゴリ別の語句数 ===")
            stats = self.manager.get_category_stats()
            stats = stats.sort_values('語句数', ascending=False)
            display(stats)
            
            # 可視化
            plt.figure(figsize=(10, 8))
            sns.barplot(x='語句数', y='カテゴリ', data=stats)
            plt.title('カテゴリ別の語句数')
            plt.tight_layout()
            plt.show()
    
    def on_save_button_clicked(self, b):
        with self.overview_output:
            try:
                self.manager.save_categories()
                print(f"{len(self.manager.categories)}個のカテゴリを '{self.manager.file_path}' に保存しました。")
            except Exception as e:
                print(f"保存中にエラーが発生しました: {e}")
    
    def on_add_button_clicked(self, b):
        name = self.new_category_name.value.strip()
        pattern = self.new_category_pattern.value.strip()
        
        with self.add_output:
            clear_output()
            
            if not name or not pattern:
                print("カテゴリ名とパターンを入力してください。")
                return
            
            # カテゴリが既に存在するか確認
            if name in self.manager.categories:
                print(f"カテゴリ '{name}' は既に存在します。上書きしますか？")
                
                confirm_button = widgets.Button(
                    description='上書き',
                    button_style='warning',
                    icon='check'
                )
                
                cancel_button = widgets.Button(
                    description='キャンセル',
                    button_style='danger',
                    icon='times'
                )
                
                def on_confirm(b):
                    success = self.manager.add_category(name, pattern)
                    with self.add_output:
                        clear_output()
                        if success:
                            print(f"カテゴリ '{name}' を上書きしました。")
                            self.update_category_dropdown()
                        else:
                            print("カテゴリの追加に失敗しました。")
                
                def on_cancel(b):
                    with self.add_output:
                        clear_output()
                        print("上書きをキャンセルしました。")
                
                confirm_button.on_click(on_confirm)
                cancel_button.on_click(on_cancel)
                
                display(widgets.HBox([confirm_button, cancel_button]))
            else:
                success = self.manager.add_category(name, pattern)
                if success:
                    print(f"カテゴリ '{name}' を追加しました。")
                    # 入力フィールドをクリア
                    self.new_category_name.value = ""
                    self.new_category_pattern.value = ""
                    self.update_category_dropdown()
                else:
                    print("カテゴリの追加に失敗しました。")
    
    def update_category_dropdown(self):
        # カテゴリのリストを取得
        categories = list(self.manager.categories.keys())
        
        # ドロップダウンを更新
        self.category_dropdown.options = categories
        if categories:
            self.category_dropdown.value = categories[0]
    
    def on_load_edit_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        
        # カテゴリ名を設定
        self.edit_category_name.value = category
        
        # 語句一覧を更新
        self.update_terms_list()
        
        with self.edit_output:
            clear_output()
            print(f"カテゴリ '{category}' を読み込みました。")
    
    def update_terms_list(self):
        if not self.category_dropdown.value:
            return
        
        category = self.category_dropdown.value
        
        with self.terms_list_output:
            clear_output()
            
            # 語句の一覧を表示
            terms = self.manager.get_category_terms(category)
            print(f"登録されている語句数: {len(terms)}")
            
            if len(terms) > 0:
                # 語句をHTMLで折り返し表示
                html_content = "<div style='word-wrap: break-word; white-space: normal; width: 100%; margin-top: 10px;'>"
                term_list = "、".join(terms)
                html_content += term_list
                html_content += "</div>"
                
                display(widgets.HTML(html_content))
    
    def on_update_name_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        old_name = self.category_dropdown.value
        new_name = self.edit_category_name.value.strip()
        
        with self.edit_output:
            clear_output()
            
            if not new_name:
                print("新しいカテゴリ名を入力してください。")
                return
            
            success = self.manager.update_category(old_name, new_name)
            if success:
                print(f"カテゴリ名を更新しました: '{old_name}' → '{new_name}'")
                self.update_category_dropdown()
                
                # ドロップダウンの選択を新しいカテゴリ名に変更
                if new_name in self.category_dropdown.options:
                    self.category_dropdown.value = new_name
                    self.update_terms_list()
            else:
                print("カテゴリ名の更新に失敗しました。")
    
    def on_add_terms_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        terms = self.edit_terms_textarea.value.strip()
        
        with self.edit_output:
            clear_output()
            
            if not terms:
                print("追加する語句を入力してください。")
                return
            
            success = self.manager.add_terms_to_category(category, terms)
            if success:
                print(f"カテゴリ '{category}' に語句を追加しました。")
                
                # 追加した語句を特定
                new_terms = terms.split('|')
                
                print("追加した語句:")
                print("  " + "、".join(new_terms))
                
                # 語句一覧を更新
                self.update_terms_list()
                
                # 入力欄をクリア
                self.edit_terms_textarea.value = ""
            else:
                print("語句の追加に失敗しました。")
    
    def on_delete_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        
        with self.edit_output:
            clear_output()
            print(f"カテゴリ '{category}' を削除しますか？")
            
            confirm_button = widgets.Button(
                description='削除',
                button_style='danger',
                icon='check'
            )
            
            cancel_button = widgets.Button(
                description='キャンセル',
                button_style='info',
                icon='times'
            )
            
            def on_confirm(b):
                success = self.manager.delete_category(category)
                with self.edit_output:
                    clear_output()
                    if success:
                        print(f"カテゴリ '{category}' を削除しました。")
                        self.update_category_dropdown()
                        
                        # 語句一覧をクリア
                        with self.terms_list_output:
                            clear_output()
                    else:
                        print("カテゴリの削除に失敗しました。")
            
            def on_cancel(b):
                with self.edit_output:
                    clear_output()
                    print("削除をキャンセルしました。")
            
            confirm_button.on_click(on_confirm)
            cancel_button.on_click(on_cancel)
            
            display(widgets.HBox([confirm_button, cancel_button]))
    
    def on_test_button_clicked(self, b):
        text = self.test_text.value.strip()
        
        with self.test_output:
            clear_output()
            
            if not text:
                print("テストするテキストを入力してください。")
                return
            
            results = self.manager.test_pattern(text)
            matched = [cat for cat, match, _ in results if match]
            
            print(f"テキスト: '{text}'")
            
            if matched:
                print(f"\n一致したカテゴリ: {len(matched)}個")
                for cat, match, match_text in results:
                    if match:
                        print(f"✓ {cat}: {match_text}")
                
                print(f"\n分類結果: {self.manager.classify(text)}")
            else:
                print("どのカテゴリにも一致しませんでした。「その他」に分類されます。")
    
    def on_export_button_clicked(self, b):
        file_path = self.csv_file_path.value.strip()
        
        with self.import_export_output:
            clear_output()
            
            if not file_path:
                print("CSVファイル名を入力してください。")
                return
            
            try:
                df = self.manager.export_to_csv(file_path)
                print(f"{len(df)}件のデータをCSVにエクスポートしました。")
                display(df.head(10))
            except Exception as e:
                print(f"エクスポート中にエラーが発生しました: {e}")
    
    def on_import_button_clicked(self, b):
        file_path = self.csv_file_path.value.strip()
        
        with self.import_export_output:
            clear_output()
            
            if not file_path:
                print("CSVファイル名を入力してください。")
                return
            
            try:
                overwrite = self.import_mode.value == 'overwrite'
                success = self.manager.import_from_csv(file_path, overwrite)
                if success:
                    print(f"CSVからカテゴリをインポートしました。")
                    self.update_category_dropdown()
                    self.refresh_overview()
                else:
                    print("インポートに失敗しました。")
            except Exception as e:
                print(f"インポート中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
                
    def on_classify_button_clicked(self, b):
        input_file = self.input_csv_file.value.strip()
        column = self.disease_column.value.strip()
        output_file = self.output_csv_file.value.strip()
        
        with self.batch_output:
            clear_output()
            
            if not input_file or not column or not output_file:
                print("入力ファイル、傷病名の列、出力ファイルを指定してください。")
                return
            
            try:
                print(f"CSVファイル '{input_file}' を読み込んでいます...")
                df = pd.read_csv(input_file, encoding='utf-8')
                
                if column not in df.columns:
                    print(f"列 '{column}' がCSVファイルに存在しません。")
                    print(f"利用可能な列: {', '.join(df.columns)}")
                    return
                
                print(f"{len(df)}件のデータを読み込みました。分類を開始します...")
                
                # 前処理関数
                def preprocess_disease_name(name):
                    if pd.isna(name):
                        return "不明"
                    if isinstance(name, (int, float)):
                        return str(int(name))
                    return str(name)
                
                # 前処理
                df[column] = df[column].apply(preprocess_disease_name)
                
                # 正規化
                df['normalized_name'] = df[column].apply(self.manager.normalize_disease_name)
                
                # 分類
                df['category'] = df['normalized_name'].apply(self.manager.classify)
                
                # カテゴリごとの集計
                category_counts = df['category'].value_counts()
                
                # 結果の表示
                print("\n=== 分類結果 ===")
                print("\n各カテゴリの件数:")
                for category, count in category_counts.items():
                    print(f"{category}: {count:,}件")
                
                # API呼び出し必要数の計算
                api_calls_needed = int(category_counts.get('その他', 0))
                api_calls_percentage = (api_calls_needed / len(df)) * 100
                
                print(f"\n「その他」に分類された件数: {api_calls_needed:,}件")
                print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")
                
                # 可視化
                plt.figure(figsize=(12, 8))
                sns.barplot(x=category_counts.values, y=category_counts.index)
                plt.title('傷病名カテゴリ別の件数')
                plt.xlabel('件数')
                plt.ylabel('カテゴリ')
                plt.tight_layout()
                plt.show()
                
                # その他に分類された例の確認
                if api_calls_needed > 0:
                    print("\n=== 「その他」に分類された傷病名の例（先頭20件）===")
                    other_examples = df[df['category'] == 'その他'][column].value_counts().head(20)
                    for disease, count in other_examples.items():
                        print(f"{disease}: {count:,}件")
                
                # CSV出力
                df.to_csv(output_file, encoding='utf-8', index=False)
                print(f"\n分類結果を '{output_file}' に保存しました。")
                
                # データを保持 (統計用)
                self.manager.data_df = df
                self.manager.last_csv_path = input_file
                self.manager.last_column_name = column
                
            except Exception as e:
                print(f"分類処理中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
    
    def on_classify_button_clicked(self, b):
        input_file = self.input_csv_file.value.strip()
        column = self.disease_column.value.strip()
        output_file = self.output_csv_file.value.strip()
        
        with self.batch_output:
            clear_output()
            
            if not input_file or not column or not output_file:
                print("入力ファイル、傷病名の列、出力ファイルを指定してください。")
                return
            
            try:
                print(f"CSVファイル '{input_file}' を読み込んでいます...")
                df = pd.read_csv(input_file, encoding='utf-8')
                
                if column not in df.columns:
                    print(f"列 '{column}' がCSVファイルに存在しません。")
                    print(f"利用可能な列: {', '.join(df.columns)}")
                    return
                
                print(f"{len(df)}件のデータを読み込みました。分類を開始します...")
                
                # 前処理関数
                def preprocess_disease_name(name):
                    if pd.isna(name):
                        return "不明"
                    if isinstance(name, (int, float)):
                        return str(int(name))
                    return str(name)
                
                # 前処理
                df[column] = df[column].apply(preprocess_disease_name)
                
                # 正規化
                df['normalized_name'] = df[column].apply(self.manager.normalize_disease_name)
                
                # 分類
                df['category'] = df['normalized_name'].apply(self.manager.classify)
                
                # カテゴリごとの集計
                category_counts = df['category'].value_counts()
                
                # 結果の表示
                print("\n=== 分類結果 ===")
                print("\n各カテゴリの件数:")
                for category, count in category_counts.items():
                    print(f"{category}: {count:,}件")
                
                # API呼び出し必要数の計算
                api_calls_needed = int(category_counts.get('その他', 0))
                api_calls_percentage = (api_calls_needed / len(df)) * 100
                
                print(f"\n「その他」に分類された件数: {api_calls_needed:,}件")
                print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")
                
                # 可視化
                plt.figure(figsize=(12, 8))
                sns.barplot(x=category_counts.values, y=category_counts.index)
                plt.title('傷病名カテゴリ別の件数')
                plt.xlabel('件数')
                plt.ylabel('カテゴリ')
                plt.tight_layout()
                plt.show()
                
                # その他に分類された例の確認
                if api_calls_needed > 0:
                    print("\n=== 「その他」に分類された傷病名の例（先頭20件）===")
                    other_examples = df[df['category'] == 'その他'][column].value_counts().head(20)
                    for disease, count in other_examples.items():
                        print(f"{disease}: {count:,}件")
                
                # CSV出力
                df.to_csv(output_file, encoding='utf-8', index=False)
                print(f"\n分類結果を '{output_file}' に保存しました。")
                
                # データを保持 (統計用)
                self.manager.data_df = df
                self.manager.last_csv_path = input_file
                self.manager.last_column_name = column
                
            except Exception as e:
                print(f"分類処理中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc(), '', text)
        
        return text.strip()
    
    def export_to_csv(self, file_path):
        """カテゴリとパターンをCSVに出力する"""
        result = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            for term in terms:
                result.append({"カテゴリ": category, "語句": term})
        
        df = pd.DataFrame(result)
        df.to_csv(file_path, encoding='utf-8', index=False)
        print(f"カテゴリと語句を '{file_path}' に出力しました。")
        return df
    
    def import_from_csv(self, file_path, overwrite=False):
        """CSVからカテゴリとパターンを読み込む"""
        df = pd.read_csv(file_path, encoding='utf-8')
        
        if "カテゴリ" not in df.columns or "語句" not in df.columns:
            print("CSVファイルには 'カテゴリ' と '語句' の列が必要です。")
            return False
        
        # カテゴリごとに語句をグループ化
        new_categories = {}
        for category, group in df.groupby("カテゴリ"):
            terms = group["語句"].tolist()
            pattern = "(" + "|".join(terms) + ")"
            try:
                re.compile(pattern)
                new_categories[category] = pattern
            except re.error as e:
                print(f"カテゴリ '{category}' の正規表現エラー: {e}")
        
        if overwrite:
            self.categories = new_categories
        else:
            self.categories.update(new_categories)
        
        self._compile_patterns()
        print(f"{len(new_categories)}個のカテゴリを '{file_path}' から読み込みました。")
        return True
    
    def get_category_terms(self, name):
        """カテゴリに含まれる語句のリストを取得する（元の順序を維持）"""
        if name not in self.categories:
            return []
        
        pattern = self.categories[name]
        # 括弧を削除し、パイプで区切って語句のリストを取得
        terms = pattern.strip("()").split("|")
        return terms
    
    def get_category_stats(self):
        """カテゴリごとの語句数などの統計情報を取得する"""
        stats = []
        for category, pattern in self.categories.items():
            terms = pattern.strip("()").split("|")
            stats.append({
                "カテゴリ": category,
                "語句数": len(terms),
                "パターン長": len(pattern)
            })
        
        return pd.DataFrame(stats)


# 簡素化されたインターフェースの実装
class CategoryManagerUI:
    def __init__(self, json_path='disease_categories.json'):
        self.manager = CategoryManager(json_path)
        self.setup_ui()
        
    def setup_ui(self):
        # メインタブを作成
        self.tab = widgets.Tab()
        
        # 各タブの内容を作成
        self.setup_overview_tab()
        self.setup_add_tab()
        self.setup_edit_tab()
        self.setup_test_tab()
        self.setup_import_export_tab()
        
        # タブの設定
        self.tab.children = [
            self.overview_tab,
            self.add_tab,
            self.edit_tab,
            self.test_tab,
            self.import_export_tab
        ]
        
        # タブのタイトル設定
        self.tab.set_title(0, '概要')
        self.tab.set_title(1, 'カテゴリ追加')
        self.tab.set_title(2, 'カテゴリ編集')
        self.tab.set_title(3, 'テスト')
        self.tab.set_title(4, 'インポート/エクスポート')
        
        # UIを表示
        display(self.tab)
        self.refresh_overview()
    
    def setup_overview_tab(self):
        # 更新ボタン
        self.refresh_button = widgets.Button(
            description='更新',
            button_style='info',
            icon='refresh'
        )
        self.refresh_button.on_click(self.on_refresh_button_clicked)
        
        # 保存ボタン
        self.save_button = widgets.Button(
            description='保存',
            button_style='success',
            icon='save'
        )
        self.save_button.on_click(self.on_save_button_clicked)
        
        # 出力エリア
        self.overview_output = widgets.Output()
        
        # タブのレイアウト
        self.overview_tab = widgets.VBox([
            widgets.HBox([self.refresh_button, self.save_button]),
            self.overview_output
        ])
    
    def setup_add_tab(self):
        # 入力フィールド
        self.new_category_name = widgets.Text(
            description='カテゴリ名:',
            style={'description_width': 'initial'}
        )
        
        self.new_category_pattern = widgets.Textarea(
            description='パターン:',
            placeholder='例: (風邪|感冒|発熱)',
            style={'description_width': 'initial'}
        )
        
        # 追加ボタン
        self.add_button = widgets.Button(
            description='追加',
            button_style='success',
            icon='plus'
        )
        self.add_button.on_click(self.on_add_button_clicked)
        
        # 出力エリア
        self.add_output = widgets.Output()
        
        # タブのレイアウト
        self.add_tab = widgets.VBox([
            self.new_category_name,
            self.new_category_pattern,
            self.add_button,
            self.add_output
        ])
    
    def setup_edit_tab(self):
        # カテゴリ選択ドロップダウン
        self.category_dropdown = widgets.Dropdown(
            description='カテゴリ:',
            style={'description_width': 'initial'}
        )
        
        # 読み込みボタン
        self.load_edit_button = widgets.Button(
            description='読み込み',
            button_style='info',
            icon='download'
        )
        self.load_edit_button.on_click(self.on_load_edit_button_clicked)
        
        # カテゴリ名変更
        self.edit_category_name = widgets.Text(
            description='新カテゴリ名:',
            style={'description_width': 'initial'}
        )
        
        self.update_name_button = widgets.Button(
            description='名前変更',
            button_style='warning',
            icon='edit'
        )
        self.update_name_button.on_click(self.on_update_name_button_clicked)
        
        # 語句追加
        self.edit_terms_textarea = widgets.Textarea(
            description='追加する語句:',
            placeholder='例: 喘鳴|鼻汁|鼻閉|くしゃみ（|で区切って入力）',
            style={'description_width': 'initial'}
        )
        
        self.add_terms_button = widgets.Button(
            description='語句追加',
            button_style='success',
            icon='plus'
        )
        self.add_terms_button.on_click(self.on_add_terms_button_clicked)
        
        # 削除ボタン
        self.delete_button = widgets.Button(
            description='削除',
            button_style='danger',
            icon='trash'
        )
        self.delete_button.on_click(self.on_delete_button_clicked)
        
        # 語句一覧表示エリア
        self.terms_list_output = widgets.Output()
        
        # 出力エリア
        self.edit_output = widgets.Output()
        
        # タブのレイアウト
        self.edit_tab = widgets.VBox([
            widgets.HBox([self.category_dropdown, self.load_edit_button]),
            widgets.HTML("<h4>カテゴリ名の変更</h4>"),
            widgets.HBox([self.edit_category_name, self.update_name_button]),
            widgets.HTML("<h4>語句の追加</h4>"),
            self.edit_terms_textarea,
            self.add_terms_button,
            widgets.HTML("<h4>カテゴリの削除</h4>"),
            self.delete_button,
            widgets.HTML("<h4>登録語句一覧</h4>"),
            self.terms_list_output,
            self.edit_output
        ])
        
        # カテゴリリストを更新
        self.update_category_dropdown()
    
    def setup_test_tab(self):
        # テストテキスト入力
        self.test_text = widgets.Text(
            value='',
            placeholder='例: 頭痛、発熱、咳、腹痛など',
            description='テストテキスト:',
            style={'description_width': 'initial'}
        )
        
        # テストボタン
        self.test_button = widgets.Button(
            description='テスト',
            button_style='info',
            icon='search'
        )
        self.test_button.on_click(self.on_test_button_clicked)
        
        # 出力エリア
        self.test_output = widgets.Output()
        
        # タブのレイアウト
        self.test_tab = widgets.VBox([
            self.test_text,
            self.test_button,
            self.test_output
        ])
    
    def setup_import_export_tab(self):
        # ファイル名入力
        self.csv_file_path = widgets.Text(
            value='disease_categories.csv',
            description='CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # エクスポートボタン
        self.export_button = widgets.Button(
            description='エクスポート',
            button_style='primary',
            icon='file-export'
        )
        self.export_button.on_click(self.on_export_button_clicked)
        
        # インポートボタン
        self.import_button = widgets.Button(
            description='インポート',
            button_style='info',
            icon='file-import'
        )
        self.import_button.on_click(self.on_import_button_clicked)
        
        # インポート方法
        self.import_mode = widgets.RadioButtons(
            options=[('新規カテゴリのみ追加', 'add'), ('すべて上書き', 'overwrite')],
            value='add',
            description='インポート方法:',
            style={'description_width': 'initial'}
        )
        
        # 出力エリア
        self.import_export_output = widgets.Output()
        
        # バッチ処理セクション
        self.batch_title = widgets.HTML("<h3>CSVファイルの傷病名を一括分類</h3>")
        
        # 入力CSVファイル
        self.input_csv_file = widgets.Text(
            value='',
            placeholder='例: 傷病名一覧.csv',
            description='入力CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # 傷病名の列名
        self.disease_column = widgets.Text(
            value='傷病名',
            description='傷病名の列:',
            style={'description_width': 'initial'}
        )
        
        # 出力CSVファイル名
        self.output_csv_file = widgets.Text(
            value='classified_diseases.csv',
            description='出力CSVファイル:',
            style={'description_width': 'initial'}
        )
        
        # 分類実行ボタン
        self.classify_button = widgets.Button(
            description='分類実行',
            button_style='primary',
            icon='play'
        )
        self.classify_button.on_click(self.on_classify_button_clicked)
        
        # バッチ処理の出力エリア
        self.batch_output = widgets.Output()
        
        # タブのレイアウト
        self.import_export_tab = widgets.VBox([
            widgets.HTML("<h3>カテゴリの入出力</h3>"),
            self.csv_file_path,
            widgets.HBox([self.export_button, self.import_button]),
            self.import_mode,
            self.import_export_output,
            widgets.HTML("<hr>"),
            self.batch_title,
            self.input_csv_file,
            self.disease_column,
            self.output_csv_file,
            self.classify_button,
            self.batch_output
        ])
    
    # ========== イベントハンドラ ==========
    def on_refresh_button_clicked(self, b):
        self.refresh_overview()
    
    def refresh_overview(self):
        with self.overview_output:
            clear_output()
            
            # カテゴリ数を表示
            categories_count = len(self.manager.categories)
            print(f"現在のファイル: {self.manager.file_path}")
            print(f"{categories_count}個のカテゴリが登録されています。")
            
            # 語句の総数を計算
            total_terms = 0
            for pattern in self.manager.categories.values():
                terms = pattern.strip("()").split("|")
                total_terms += len(terms)
            
            print(f"登録されている語句の総数: {total_terms}語")
            
            # 各カテゴリの語句数の統計
            print("\n=== カテゴリ別の語句数 ===")
            stats = self.manager.get_category_stats()
            stats = stats.sort_values('語句数', ascending=False)
            display(stats)
            
            # 可視化
            plt.figure(figsize=(10, 8))
            sns.barplot(x='語句数', y='カテゴリ', data=stats)
            plt.title('カテゴリ別の語句数')
            plt.tight_layout()
            plt.show()
    
    def on_save_button_clicked(self, b):
        with self.overview_output:
            try:
                self.manager.save_categories()
                print(f"{len(self.manager.categories)}個のカテゴリを '{self.manager.file_path}' に保存しました。")
            except Exception as e:
                print(f"保存中にエラーが発生しました: {e}")
    
    def on_add_button_clicked(self, b):
        name = self.new_category_name.value.strip()
        pattern = self.new_category_pattern.value.strip()
        
        with self.add_output:
            clear_output()
            
            if not name or not pattern:
                print("カテゴリ名とパターンを入力してください。")
                return
            
            # カテゴリが既に存在するか確認
            if name in self.manager.categories:
                print(f"カテゴリ '{name}' は既に存在します。上書きしますか？")
                
                confirm_button = widgets.Button(
                    description='上書き',
                    button_style='warning',
                    icon='check'
                )
                
                cancel_button = widgets.Button(
                    description='キャンセル',
                    button_style='danger',
                    icon='times'
                )
                
                def on_confirm(b):
                    success = self.manager.add_category(name, pattern)
                    with self.add_output:
                        clear_output()
                        if success:
                            print(f"カテゴリ '{name}' を上書きしました。")
                            self.update_category_dropdown()
                        else:
                            print("カテゴリの追加に失敗しました。")
                
                def on_cancel(b):
                    with self.add_output:
                        clear_output()
                        print("上書きをキャンセルしました。")
                
                confirm_button.on_click(on_confirm)
                cancel_button.on_click(on_cancel)
                
                display(widgets.HBox([confirm_button, cancel_button]))
            else:
                success = self.manager.add_category(name, pattern)
                if success:
                    print(f"カテゴリ '{name}' を追加しました。")
                    # 入力フィールドをクリア
                    self.new_category_name.value = ""
                    self.new_category_pattern.value = ""
                    self.update_category_dropdown()
                else:
                    print("カテゴリの追加に失敗しました。")
    
    def update_category_dropdown(self):
        # カテゴリのリストを取得
        categories = list(self.manager.categories.keys())
        
        # ドロップダウンを更新
        self.category_dropdown.options = categories
        if categories:
            self.category_dropdown.value = categories[0]
    
    def on_load_edit_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        
        # カテゴリ名を設定
        self.edit_category_name.value = category
        
        # 語句一覧を更新
        self.update_terms_list()
        
        with self.edit_output:
            clear_output()
            print(f"カテゴリ '{category}' を読み込みました。")
    
    def update_terms_list(self):
        if not self.category_dropdown.value:
            return
        
        category = self.category_dropdown.value
        
        with self.terms_list_output:
            clear_output()
            
            # 語句の一覧を表示
            terms = self.manager.get_category_terms(category)
            print(f"登録されている語句数: {len(terms)}")
            
            if len(terms) > 0:
                # 語句をHTMLで折り返し表示
                html_content = "<div style='word-wrap: break-word; white-space: normal; width: 100%; margin-top: 10px;'>"
                term_list = "、".join(terms)
                html_content += term_list
                html_content += "</div>"
                
                display(widgets.HTML(html_content))
    
    def on_update_name_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        old_name = self.category_dropdown.value
        new_name = self.edit_category_name.value.strip()
        
        with self.edit_output:
            clear_output()
            
            if not new_name:
                print("新しいカテゴリ名を入力してください。")
                return
            
            success = self.manager.update_category(old_name, new_name)
            if success:
                print(f"カテゴリ名を更新しました: '{old_name}' → '{new_name}'")
                self.update_category_dropdown()
                
                # ドロップダウンの選択を新しいカテゴリ名に変更
                if new_name in self.category_dropdown.options:
                    self.category_dropdown.value = new_name
                    self.update_terms_list()
            else:
                print("カテゴリ名の更新に失敗しました。")
    
    def on_add_terms_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        terms = self.edit_terms_textarea.value.strip()
        
        with self.edit_output:
            clear_output()
            
            if not terms:
                print("追加する語句を入力してください。")
                return
            
            success = self.manager.add_terms_to_category(category, terms)
            if success:
                print(f"カテゴリ '{category}' に語句を追加しました。")
                
                # 追加した語句を特定
                new_terms = terms.split('|')
                
                print("追加した語句:")
                print("  " + "、".join(new_terms))
                
                # 語句一覧を更新
                self.update_terms_list()
                
                # 入力欄をクリア
                self.edit_terms_textarea.value = ""
            else:
                print("語句の追加に失敗しました。")
    
    def on_delete_button_clicked(self, b):
        if not self.category_dropdown.value:
            with self.edit_output:
                clear_output()
                print("カテゴリを選択してください。")
            return
        
        category = self.category_dropdown.value
        
        with self.edit_output:
            clear_output()
            print(f"カテゴリ '{category}' を削除しますか？")
            
            confirm_button = widgets.Button(
                description='削除',
                button_style='danger',
                icon='check'
            )
            
            cancel_button = widgets.Button(
                description='キャンセル',
                button_style='info',
                icon='times'
            )
            
            def on_confirm(b):
                success = self.manager.delete_category(category)
                with self.edit_output:
                    clear_output()
                    if success:
                        print(f"カテゴリ '{category}' を削除しました。")
                        self.update_category_dropdown()
                        
                        # 語句一覧をクリア
                        with self.terms_list_output:
                            clear_output()
                    else:
                        print("カテゴリの削除に失敗しました。")
            
            def on_cancel(b):
                with self.edit_output:
                    clear_output()
                    print("削除をキャンセルしました。")
            
            confirm_button.on_click(on_confirm)
            cancel_button.on_click(on_cancel)
            
            display(widgets.HBox([confirm_button, cancel_button]))
    
    def on_test_button_clicked(self, b):
        text = self.test_text.value.strip()
        
        with self.test_output:
            clear_output()
            
            if not text:
                print("テストするテキストを入力してください。")
                return
            
            results = self.manager.test_pattern(text)
            matched = [cat for cat, match, _ in results if match]
            
            print(f"テキスト: '{text}'")
            
            if matched:
                print(f"\n一致したカテゴリ: {len(matched)}個")
                for cat, match, match_text in results:
                    if match:
                        print(f"✓ {cat}: {match_text}")
                
                print(f"\n分類結果: {self.manager.classify(text)}")
            else:
                print("どのカテゴリにも一致しませんでした。「その他」に分類されます。")
    
    def on_export_button_clicked(self, b):
        file_path = self.csv_file_path.value.strip()
        
        with self.import_export_output:
            clear_output()
            
            if not file_path:
                print("CSVファイル名を入力してください。")
                return
            
            try:
                df = self.manager.export_to_csv(file_path)
                print(f"{len(df)}件のデータをCSVにエクスポートしました。")
                display(df.head(10))
            except Exception as e:
                print(f"エクスポート中にエラーが発生しました: {e}")
    
    def on_import_button_clicked(self, b):
        file_path = self.csv_file_path.value.strip()
        
        with self.import_export_output:
            clear_output()
            
            if not file_path:
                print("CSVファイル名を入力してください。")
                return
            
            try:
                overwrite = self.import_mode.value == 'overwrite'
                success = self.manager.import_from_csv(file_path, overwrite)
                if success:
                    print(f"CSVからカテゴリをインポートしました。")
                    self.update_category_dropdown()
                    self.refresh_overview()
                else:
                    print("インポートに失敗しました。")
            except Exception as e:
                print(f"インポート中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc()
    
    def on_classify_button_clicked(self, b):
        input_file = self.input_csv_file.value.strip()
        column = self.disease_column.value.strip()
        output_file = self.output_csv_file.value.strip()
        
        with self.batch_output:
            clear_output()
            
            if not input_file or not column or not output_file:
                print("入力ファイル、傷病名の列、出力ファイルを指定してください。")
                return
            
            try:
                print(f"CSVファイル '{input_file}' を読み込んでいます...")
                df = pd.read_csv(input_file, encoding='utf-8')
                
                if column not in df.columns:
                    print(f"列 '{column}' がCSVファイルに存在しません。")
                    print(f"利用可能な列: {', '.join(df.columns)}")
                    return
                
                print(f"{len(df)}件のデータを読み込みました。分類を開始します...")
                
                # 前処理関数
                def preprocess_disease_name(name):
                    if pd.isna(name):
                        return "不明"
                    if isinstance(name, (int, float)):
                        return str(int(name))
                    return str(name)
                
                # 前処理
                df[column] = df[column].apply(preprocess_disease_name)
                
                # 正規化
                df['normalized_name'] = df[column].apply(self.manager.normalize_disease_name)
                
                # 分類
                df['category'] = df['normalized_name'].apply(self.manager.classify)
                
                # カテゴリごとの集計
                category_counts = df['category'].value_counts()
                
                # 結果の表示
                print("\n=== 分類結果 ===")
                print("\n各カテゴリの件数:")
                for category, count in category_counts.items():
                    print(f"{category}: {count:,}件")
                
                # API呼び出し必要数の計算
                api_calls_needed = int(category_counts.get('その他', 0))
                api_calls_percentage = (api_calls_needed / len(df)) * 100
                
                print(f"\n「その他」に分類された件数: {api_calls_needed:,}件")
                print(f"全サンプルに対する割合: {api_calls_percentage:.2f}%")
                
                # 可視化
                plt.figure(figsize=(12, 8))
                sns.barplot(x=category_counts.values, y=category_counts.index)
                plt.title('傷病名カテゴリ別の件数')
                plt.xlabel('件数')
                plt.ylabel('カテゴリ')
                plt.tight_layout()
                plt.show()
                
                # その他に分類された例の確認
                if api_calls_needed > 0:
                    print("\n=== 「その他」に分類された傷病名の例（先頭20件）===")
                    other_examples = df[df['category'] == 'その他'][column].value_counts().head(20)
                    for disease, count in other_examples.items():
                        print(f"{disease}: {count:,}件")
                
                # CSV出力
                df.to_csv(output_file, encoding='utf-8', index=False)
                print(f"\n分類結果を '{output_file}' に保存しました。")
                
                # データを保持 (統計用)
                self.manager.data_df = df
                self.manager.last_csv_path = input_file
                self.manager.last_column_name = column
                
            except Exception as e:
                print(f"分類処理中にエラーが発生しました: {e}")
                import traceback
                traceback.print_exc()

IndentationError: unexpected indent (940207116.py, line 43)